# Deploy Spark

In [1]:
import pprint, time, sys

In [2]:
varClouderaPath = "/vagrant/notebooks/cloudera"
sys.path.append(varClouderaPath)

varClouderaPath

'/vagrant/notebooks/cloudera'

In [3]:
sys.path

['',
 '/opt/anaconda/lib/python27.zip',
 '/opt/anaconda/lib/python2.7',
 '/opt/anaconda/lib/python2.7/plat-linux2',
 '/opt/anaconda/lib/python2.7/lib-tk',
 '/opt/anaconda/lib/python2.7/lib-old',
 '/opt/anaconda/lib/python2.7/lib-dynload',
 '/opt/anaconda/lib/python2.7/site-packages',
 '/opt/anaconda/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/vagrant/notebooks/cloudera']

In [4]:
!free

             total       used       free     shared    buffers     cached
Mem:       3922528    2308228    1614300        164      62008    1689136
-/+ buffers/cache:     557084    3365444 
Swap:      1015804          0    1015804 


# Connect to Cloudera Manager

In [5]:
from cm_api.endpoints.services import ApiService
from cm_api.endpoints.services import ApiServiceSetupInfo
from cm_api.api_client import ApiResource

In [6]:
varFullHost = "pocd-cm581-dev-manager.poc-d.internal"

varAdminUser = "admin"
varAdminPassword = "admin"

varApiResource = ApiResource(varFullHost, version=10, username=varAdminUser, password=varAdminPassword)
varManager = varApiResource.get_cloudera_manager()
varManager

In [ ]:
# Get cluster

In [7]:
varClusterName = "POC-D Cluster"


In [8]:
varCluster = varApiResource.get_cluster(varClusterName)


In [15]:
varSPARKServiceConfig = {
  'process_groupname': "spark",
  'process_username': "spark",
  'spark_history_log_dir': "/tmp",
  'spark_jar_hdfs_path': "",
  'yarn_service': "YARN",
}

varSPARKServiceConfig

{'process_groupname': 'spark',
 'process_username': 'spark',
 'spark_history_log_dir': '/tmp',
 'spark_jar_hdfs_path': '',
 'yarn_service': 'YARN'}

In [13]:
!mkdir -p /opt/cloudera/log/spark
!chmod -R 777 /opt/cloudera/log/spark


In [14]:
varSPARK_History_Config = {
    'history_server_max_heapsize': 268435456,
    'log_dir': "/opt/cloudera/log/spark",
    'max_log_backup_index': 10,
    'max_log_size': 100,
    'log_threshold': "INFO",
}

varSPARK_History_Config

{'history_server_max_heapsize': 268435456,
 'log_dir': '/opt/cloudera/log/spark',
 'log_threshold': 'INFO',
 'max_log_backup_index': 10,
 'max_log_size': 100}

In [11]:
!mkdir -p /etc/spark
!chmod -R 777 /etc/spark

In [12]:
varSPARK_Gateway_Config = {
    'client_config_root_dir': "/etc/spark",
    'spark_history_enabled': True,
}

varSPARK_Gateway_Config

{'client_config_root_dir': '/etc/spark', 'spark_history_enabled': True}

In [17]:
varManagementHost = "pocd-cm581-dev-man1.poc-d.internal"

varSparkHosts = [
    "pocd-cm581-dev-node1.poc-d.internal",
    "pocd-cm581-dev-node2.poc-d.internal",
    "pocd-cm581-dev-node3.poc-d.internal",
]

varSparkHosts

['pocd-cm581-dev-node1.poc-d.internal',
 'pocd-cm581-dev-node2.poc-d.internal',
 'pocd-cm581-dev-node3.poc-d.internal']

In [10]:

def deploy_spark(cluster, spark_service_name, spark_service_config, 
                 spark_history_host, spark_history_config, 
                 spark_gw_hosts, spark_gw_config):
    
    spark_service = cluster.create_service(spark_service_name, "SPARK_ON_YARN")
    spark_service.update_config(spark_service_config)

    spark_service.create_role("{0}-sh".format(spark_service_name), "SPARK_YARN_HISTORY_SERVER", spark_history_host)
    
    gateway = 0
    for host in spark_gw_hosts:
        gateway += 1
        spark_service.create_role("{0}-gw-".format(spark_service_name) + str(gateway), "GATEWAY", host)

    for group in spark_service.get_all_role_config_groups():
        if group.roleType == "SPARK_YARN_HISTORY_SERVER":
            group.update_config(spark_history_config)
        elif group.roleType == "GATEWAY":
            group.update_config(spark_gw_config)
    
    #TODO - CreateSparkUserDirCommand, SparkUploadJarServiceCommand???
    
    return spark_service


In [18]:
spark_service = deploy_spark(varCluster, "SPARK", varSPARKServiceConfig,
    varManagementHost, varSPARK_History_Config, 
    varSparkHosts, varSPARK_Gateway_Config)

spark_service

In [64]:
varCluster.start().wait()